# LOICA Consortia demonstration

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

This notebook shows the novel functionality that has been encoded in git branch Consortia

This LOICA update allows to create strains that belong to class Strain, taking genetic network and metabolism as parameters.

## Creating strain
First, set up metabolism and genetic network as usual

In [ ]:
def growth_rate(t):
    return 1 # gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass(t):
    return 1 # gompertz(t, 0.01, 1, 1, 0.5)

metab1 = lc.SimulatedMetabolism("Simulated metabolism 1", biomass, growth_rate)
genetic_n1 = lc.GeneticNetwork()

Then define strain

In [ ]:
test_strain = lc.Strain('Test strain', genetic_n1, metab1)
# test
print(test_strain.name)

Now, create genetic network as usual. 

Each gene product now has parameter diffusion_rate, which by default is set to 0. 

LOICA Consortia assumes that:
- all molecules are distrubed equally both incracellularly and extracellularly, so the only diffusion gradient exists between extracellular space and inside of the cell. 
- all cells belonging to the same strain are of uniform size. 

Based on that, diffusion rate here is the proportion of concentration/molecule difference that gets transferred either in- or outside of the cell per change of time.

In [ ]:
# Create and add reporter to genetic network
reporter = lc.Reporter(name='GFP', color='green', degradation_rate=0, init_concentration=0)
genetic_n1.add_reporter(reporter)

# Constitutive exression of reporter from strain
source = lc.Source(output=reporter, rate=1000)
genetic_n1.add_operator(source)

Now we have a strain that constitutively expresses GFP. 

Since we have not set diffusion rate, if we create sample, assay and run the latter then check the reporter extracellular concentration, it will stay 0:

In [ ]:
sample = lc.Sample(strain=test_strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='GFP constittive expression',
                description='Simulated GFP expression in test strain'
                )

Setting parameter track_all to true will record both intracellular and extracellular concentrations of all GeneProducts in the systems, as well as biomass without accounting for noise and background.

In [ ]:
assay.run(stochastic=False, track_all=True)

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Test strain'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Test strain biomass'].plot(x='Time', y='Measurement', style='-', ax=ax)
plt.legend(['Extracellular GFP', 'Internal GFP', 'Biomass'])

Now let's add diffusible signal C14 with diffusion rate 1 that would also be continiously expressed.

In [ ]:
regulator = lc.Regulator(name='C14', degradation_rate=0, init_concentration=0, diffusion_rate=1)
genetic_n1.add_regulator(regulator)

source_2 = lc.Source(output=regulator, rate=1000)
genetic_n1.add_operator(source_2)

In [ ]:
sample = lc.Sample(strain=test_strain)
assay2 = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='GFP and C14 constittive expression',
                description='Simulated GFP and C14 expression in test strain'
                )

In [ ]:
assay2.run(stochastic=False, track_all=True)

In [ ]:
m = assay2.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Test strain'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='C14 in Test strain'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Test strain biomass'].plot(x='Time', y='Measurement', style='-', ax=ax)
plt.legend(['Extracellular GFP', 'Internal GFP', 'Extracellular C14', 'C14 in Test strain', 'Biomass'])

It is also possible to set extracellular degradation for any GeneProduct

In [ ]:
sample.set_extracel_degr("C14", 0.5)
assay3 = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='GFP and C14 constittive expression with C14 extracellular degradation',
                description='Simulated GFP and C14 expression in test strain'
                )

In [ ]:
assay3.run(stochastic=False, track_all=True)

In [ ]:
m = assay3.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Test strain'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='C14 in Test strain'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Test strain biomass'].plot(x='Time', y='Measurement', style='-', ax=ax)
plt.legend(['Extracellular GFP', 'Internal GFP', 'Extracellular C14', 'C14 in Test strain', 'Biomass'])